In [41]:
import requests
from bs4 import BeautifulSoup
import time, random
import re
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_0) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
}
years=["2025","2024","2023","2022","2021","2020"]
list_href_player=[]
base_url = "https://www.basketball-reference.com"
for year in years:

    best_player_url="/leagues/NBA_" +year+"_totals.html"
    full_url=base_url+best_player_url

    response = requests.get(full_url, headers=headers)
    response.encoding = "utf-8"
    print(response.status_code)
    time.sleep(random.uniform(3,5))
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.select("tbody tr")

    year_int=int(year)
    for row in rows:
        rank = row.select("th")
        player_rank = rank[0].text
        numbers = "".join(re.findall(r"\d+", player_rank))
        number_int = int(numbers)

        player_cell = row.select("td")
        link_player = player_cell[0].select_one("a")
        age=player_cell[1].text
        age_int=int(age)
        total_point=player_cell[28].text
        total_point_int=int(total_point)
        print(link_player["href"])
        print(total_point_int)
        list_href_player.append({"year": year_int, "url_player": link_player["href"], "age": age_int, "player_rank": number_int,"total_point": total_point_int})
        if number_int==25:
            break

200
/players/g/gilgesh01.html
2484
/players/e/edwaran01.html
2177
/players/j/jokicni01.html
2071
/players/a/antetgi01.html
2036
/players/t/tatumja01.html
1932
/players/b/bookede01.html
1923
/players/y/youngtr01.html
1841
/players/h/herroty01.html
1840
/players/c/cunnica01.html
1830
/players/h/hardeja01.html
1802
/players/t/townska01.html
1759
/players/l/lavinza01.html
1724
/players/l/lavinza01.html
1007
/players/l/lavinza01.html
717
/players/g/greenja05.html
1723
/players/c/curryst01.html
1718
/players/d/derozde01.html
1710
/players/j/jamesle01.html
1710
/players/m/mitchdo01.html
1701
/players/b/brunsja01.html
1690
/players/d/duranke01.html
1647
/players/j/jacksja02.html
1641
/players/s/siakapa01.html
1578
/players/g/garlada01.html
1544
/players/w/whiteco01.html
1509
/players/w/willija06.html
1490
/players/r/reaveau01.html
1475
200
/players/d/doncilu01.html
2370
/players/g/gilgesh01.html
2254
/players/a/antetgi01.html
2222
/players/b/brunsja01.html
2212
/players/j/jokicni01.html
2085
/

In [50]:
import pandas as pd
positions = []
records=[]
height_cm=0
weight_kg=0
experience_years=0
cleaned=None
player_name=None
team_name=None
for player in list_href_player:
    response = requests.get(base_url+player["url_player"],headers=headers)
    response.encoding = "utf-8"
    info_player_soup = BeautifulSoup(response.text, "html.parser")

    url_player = player["url_player"]  # just a string
    player_id = re.search(r'([^/]+)\.html$', url_player).group(1)

    player_name_elem = info_player_soup.select_one("#meta h1")
    player_name = player_name_elem.get_text(strip=True) if player_name_elem else None
    print(player_name)

    final_clean_positions = []
    position_elem = info_player_soup.select_one("#meta p:has(strong:-soup-contains('Position'))")

    text = position_elem.get_text(" ", strip=True)
    position = text.split("Shoots:")[0].replace("Position:", "").strip()
    clean_position = position.replace(", and", ",")
    clean2_position = clean_position.replace(" and ", ",")
    positions = [p.strip() for p in clean2_position.split(",") if p.strip()]
    for p in positions:
        cleaned = p.strip()
        cleaned = cleaned.replace("▪", "")
        cleaned = cleaned.strip()
        final_clean_positions.append(cleaned)

    team_strong = info_player_soup.select_one("#meta strong:-soup-contains('Team')")
    if team_strong:
        team_link_tag = team_strong.find_next("a")
        if team_link_tag:
            team_name = team_link_tag.get_text(strip=True)
    else:
        team_name = None



    exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Experience')")
    if exp_strong:
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))

    else:
        exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Career Length')")
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))




    height_weight_elems = info_player_soup.select_one("#meta p:has(span:-soup-contains('lb'))")
    text = height_weight_elems.get_text(" ", strip=True)
    match = re.search(r"\((\d+)cm,\s*(\d+)kg\)", text)
    height_cm = int(match.group(1))
    weight_kg = int(match.group(2))

    records.append({
        "player_id": player_id,
        "player_name": player_name,
        "age": player["age"],
        "rank": player["player_rank"],
        "season": player["year"],
        "height_cm": height_cm,
        "weight_kg": weight_kg,
        "experience_years": experience_years,
        "positions": final_clean_positions,
        "team": team_name,
        "total_point": player["total_point"],
        "groupName":"TS"
    })

    time.sleep(random.uniform(3,5))
df = pd.DataFrame(records)
print(df.head())

Shai Gilgeous-Alexander
Anthony Edwards
Nikola Jokić
Giannis Antetokounmpo
Jayson Tatum
Devin Booker
Trae Young
Tyler Herro
Cade Cunningham
James Harden
Karl-Anthony Towns
Zach LaVine
Zach LaVine
Zach LaVine
Jalen Green
Stephen Curry
DeMar DeRozan
LeBron James
Donovan Mitchell
Jalen Brunson
Kevin Durant
Jaren Jackson Jr.
Pascal Siakam
Darius Garland
Coby White
Jalen Williams
Austin Reaves
Luka Dončić
Shai Gilgeous-Alexander
Giannis Antetokounmpo
Jalen Brunson
Nikola Jokić
Anthony Edwards
Kevin Durant
Jayson Tatum
De'Aaron Fox
Stephen Curry
DeMar DeRozan
Anthony Davis
Devin Booker
LeBron James
Tyrese Maxey
Paolo Banchero
Damian Lillard
Dejounte Murray
Pascal Siakam
Pascal Siakam
Pascal Siakam
Paul George
Kawhi Leonard
Jaylen Brown
Jalen Green
Mikal Bridges
Zion Williamson
Jayson Tatum
Joel Embiid
Luka Dončić
Shai Gilgeous-Alexander
Giannis Antetokounmpo
Anthony Edwards
Julius Randle
Donovan Mitchell
Trae Young
Zach LaVine
Damian Lillard
De'Aaron Fox
DeMar DeRozan
Jaylen Brown
Pascal Sia

In [51]:
df = df.drop_duplicates(subset=["season", "rank"], keep="first").reset_index(drop=True)
df

,player_id,player_name,age,rank,season,height_cm,weight_kg,experience_years,positions,team,total_point,groupName
0,gilgesh01,Shai Gilgeous-Alexander,26,1,2025,198,90,7,"[Point Guard, Shooting Guard]",Oklahoma City Thunder,2484,TS
1,edwaran01,Anthony Edwards,23,2,2025,193,102,5,[Shooting Guard],Minnesota Timberwolves,2177,TS
2,jokicni01,Nikola Jokić,29,3,2025,211,128,10,[Center],Denver Nuggets,2071,TS
3,antetgi01,Giannis Antetokounmpo,30,4,2025,211,109,12,"[Power Forward, Small Forward, Point Guard, Sh...",Milwaukee Bucks,2036,TS
4,tatumja01,Jayson Tatum,26,5,2025,203,95,8,"[Small Forward, Power Forward]",Boston Celtics,1932,TS
...,...,...,...,...,...,...,...,...,...,...,...,...
145,siakapa01,Pascal Siakam,25,21,2020,203,104,9,[Power Forward],Indiana Pacers,1371,TS
146,sextoco01,Collin Sexton,21,22,2020,190,86,7,"[Shooting Guard, Point Guard]",Charlotte Hornets,1349,TS
147,gilgesh01,Shai Gilgeous-Alexander,21,23,2020,198,90,7,"[Point Guard, Shooting Guard]",Oklahoma City Thunder,1331,TS
148,warretj01,T.J. Warren,26,24,2020,203,99,9,"[Small Forward, Power Forward]",None,1326,TS


In [52]:
df.to_csv("bestPlayer.csv", index=False)